In [1]:
# --- Imports ---
import os
import random
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.metrics import classification_report, accuracy_score
import re

In [2]:
# --- Configuration ---
MODEL_NAME = "meta-llama/Llama-3.2-1B-Instruct" 
CSV_PATH = r"C:\Users\stdFurqan\Desktop\Sindhi (politics)\split_30.csv"
TEXT_COL = "News Headlines"
LABEL_COL = "Aspect Category"
LABELS = ["Crime", "Sports", "Politics"]
SEED = 20
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# --- Reproducibility ---
def set_seed(seed=20):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
set_seed(SEED)

In [3]:
# ✅ Local path where model is stored
MODEL_PATH = r"C:\Users\stdFurqan\Downloads\lama_models_download\LAMA_3.2(1b)"

print(f"🧠 Loading model from {MODEL_PATH} ...")

# ✅ Load tokenizer from local folder
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

# ✅ Load model from local folder
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype=torch.bfloat16,   # ✅ best for 40-series
    device_map="auto"
)

model.config.pad_token_id = tokenizer.pad_token_id
model.eval()

print("✅ Model loaded successfully from local directory!")

🧠 Loading model from C:\Users\stdFurqan\Downloads\lama_models_download\LAMA_3.2(1b) ...
✅ Model loaded successfully from local directory!


In [6]:
# --- Load dataset ---
df = pd.read_csv(CSV_PATH)
# ✅ Clean up any extra spaces or tabs in column names
df.columns = df.columns.str.strip()
print(df.columns.tolist())


['Aspect Category', 'News Headlines']


In [7]:




texts = df[TEXT_COL].astype(str).tolist()
true_labels = df[LABEL_COL].astype(str).tolist()

PROMPT_TEMPLATE = (
    "You are an intelligent Sindhi language assistant. "
    "Classify the following news headline into one of these categories: Crime, Sports, or Politics based on the examples below.\n\n"
    
    "=== Politics Examples ===\n"
    "1 Text: بلوچستان ۾ ايراني سرزمين تان ٻيهر حملو، پاڪ فوج جا 4 جوان شهيد\n   Label: Politics\n"
    "2 Text: سپريم ڪورٽ ۾ قرآن پاڪ جي حافظ کي 20 واڌو نمبر ڏيڻ بابت پاڻمرادي نوٽيس ۾ جسٽس قاضي فائز عيسيٰ جي سربراهي ۾ 3 رڪني خاص بئنچ فيصلو جاري ڪيو\n   Label: Politics\n"
    "3 Text: واضح رهي ته ڪيرول ئي نه پر ڪيترين ئي عورتن اڳوڻي آمريڪي صدر تي جنسي ڏاڍائي ۾ ملوث هجڻ جا الزام لڳايا آهن.\n   Label: Politics\n"
    "4 Text: پيادل قافلي جي ڀرپور آجيان ڪري مٿن گلن جي ورکا ڪئي وئي.\n   Label: Politics\n\n"

    "=== Sports Examples ===\n"
    "5 Text: نيپال جي ڪرڪيٽ ميچ ڊي ايل ايس ميٿڊ تحت ون ڊي ۾ 9 وڪيٽن تي يو اي اي جي ٽيم کي شڪست ڏئي ڇڏي آهي\n   Label: Sports\n"
    "6 Text: هوڏانهن ٻئي سيمي فائنل ۾ وري آريانا سيبالينڪا يونان جي ماريا سڪاري کي شڪست ڏئي فائنل ۾ جڳهه ٺاهي آهي، هاڻ فائنل ۾ آريانا سيبالينڪا ۽ ايلانا ريباڪنا آمهون سامهون ٿينديون.\n   Label: Sports\n"
    "7 Text: جڏهن تھ سريلنڪا ويمن جي ٽيم 19 اوور ۾ رڳو 3 وڪيٽن جي نقصان تي هدف پورو ڪري ڪاميابي ماڻي ورتي، جن ۾ چماري اتاپٽو 33، هرشيٿا سماراوڪرما 29، ڪويشا ڊلهاري 20 رنسن سان نمايان رهيون\n   Label: Sports\n"
    "8 Text: ڪرسٽيانو رونالڊو 2023ع دوران 38 گول ڪري سڀني کان اڳيان نڪري ويو\n   Label: Sports\n\n"

    "=== Crime Examples ===\n"
    "9 Text: پر گڏيل قومن جي هڪ بيان ۾ چيو ويو آهي ته موت جي سزا ماڻيندڙن جو اصل انگ ان کان وڌيڪ آهي.\n   Label: Crime\n"
    "10 Text: هڪ عورت کي سندس ڌيءَ سميت ڏوهاري اغوا ڪري ويا.\n   Label: Crime\n"
    "11 Text: پرڏيهي ميڊيا موجب ان ڳالهه جو انڪشاف هڪ ايراني نائب وزير يونس پناهي ڪيو آهي، جنهن چيو آهي ته ايران ۾ اسڪول جي ڇوڪرين کي زهر ڏنو ويو\n   Label: Crime\n"
    "12 Text: ڳڙهي خدابخش کوسو ۾ نينگري جي قتل جو الزام.\n   Label: Crime\n\n"

    "Reply with only one word: Politics, Sports, or Crime.\n"
    "Now classify the following text in the same way.\n\n"
    "Text: {text}\n\n"
    "Label:"
)

pattern = re.compile(r"\b(Politics|Sports|Crime)\b", re.IGNORECASE)

def extract_label(output):
    m = pattern.search(output)
    return m.group(1).capitalize() if m else "Unknown"

pred_labels = []
batch_size = 4  # smaller batch to fit in T4 VRAM
for i in range(0, len(texts), batch_size):
    batch = texts[i:i+batch_size]
    prompts = [PROMPT_TEMPLATE.format(text=t) for t in batch]
    inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True).to(DEVICE)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=6,
            do_sample=False,
            temperature=0.0,
            top_p=1.0,
            pad_token_id=tokenizer.eos_token_id,
        )

    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    for prompt, full_out in zip(prompts, decoded):
        gen = full_out[len(prompt):].strip() if full_out.startswith(prompt) else full_out
        pred_labels.append(extract_label(gen))

# --- Evaluation ---
report = classification_report(true_labels, pred_labels, labels=LABELS, digits=4, zero_division=0)
acc = accuracy_score(true_labels, pred_labels)
print("\n📊 Classification Report:\n")
print(report)
print(f"Accuracy: {acc:.4f}")

C:\Users\stdFurqan\anaconda3\envs\py312\Lib\site-packages\transformers\generation\configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(



📊 Classification Report:

              precision    recall  f1-score   support

       Crime     0.5736    0.7951    0.6665      1440
      Sports     1.0000    0.0104    0.0207      1245
    Politics     0.6058    0.7853    0.6840      2282

    accuracy                         0.5939      4967
   macro avg     0.7265    0.5303    0.4570      4967
weighted avg     0.6953    0.5939    0.5126      4967

Accuracy: 0.5939
